In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv, random, os, sys, json, requests, re
import numpy as np
import pandas as pd
from openai import OpenAI
from pandas import read_csv
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

## 1. The onion list

In [ ]:
onion = pd.read_csv("onion_path1")

# concat all the csv files
# onion index 02 ~ 10
for i in range(9):
    onion_index = pd.read_csv(f"onion_path{i+2}.csv")
    onion = pd.concat([onion, onion_index], ignore_index=True)

onion = onion.sort_values(by='Date', ascending=False)
onion = onion.dropna(subset=['Title', 'Content'])
onion = onion[~onion['Content'].str.contains("Previous Slide Next Slide List slides")]
onion = onion[onion['Content'].str.count("Advertisement") < 2]
onion = onion.reset_index(drop=True)

onion = onion.iloc[:1569, :] # for matching the number of data with reddit

In [ ]:
onion_list = []
for i in range(len(onion)):
    onion_list.append('[Title]\n' + onion.loc[i].to_list()[1] + '\n' + '[Contents]\n' + onion.loc[i].to_list()[3])


onion_list_len = [len(x) for x in onion_list]
onion_list_len = pd.Series(onion_list_len)
print(onion_list_len.describe())

## 2. Reddit list

In [ ]:
reddit = pd.read_csv('reddit_path1')
reddit1 = pd.read_csv('reddit_path2', encoding='utf-8')
reddit2 = pd.read_csv('reddit_path3', encoding='utf-8')
reddit = pd.concat([reddit, reddit1, reddit2])
reddit = reddit.dropna(subset=['Title', 'Content'])
reddit = reddit.reset_index(drop=True)


reddit_list = []
for i in range(len(reddit)):
    reddit_list.append('[Title]\n' + reddit.loc[i].to_list()[1] + '\n' + '[Contents]\n' + reddit.loc[i].to_list()[3])

# statistics
reddit_list_len = [len(x) for x in reddit_list]
reddit_list_len = [len(x) for x in reddit_list]
reddit_list_len = pd.Series(reddit_list_len)
print(reddit_list_len.describe())
print(reddit.columns)

## 3. Make Test data

In [ ]:
data = []
for i in range(len(onion_list)):
    data.append([onion_list[i], 0])
for i in range(len(reddit_list)):
    data.append([reddit_list[i], 1])


print(f'The length of data: {len(data)}')

random.seed(43)
random.shuffle(data)

## 4. GPT4o

In [ ]:
prompt = """The uploaded text is one of the articles that may be real or fake.
Please Answer whether below article is fake or real. Give a 20-character rationale for why you think that way, and output a 0 and 1 at the end of the sentence.
To Solve this, You have to think step by step. The first step in identifying fake news is evaluating the reliability of the information source. Well-known and verified news organizations are generally more reliable, and their reports can be trusted more than unverified sources.
In addition to source reliability, look at the language used in the content. Fake news often uses sensational or exaggerated language designed to elicit an emotional response. It is also important to check for consistency and accuracy in the information presented; fake news typically includes claims that are either unverified or clearly false. Another critical step is cross-verification,
where check if the same claims are reported by multiple trusted sources.
i.e. rationale + answer 1 if you think the article is real, rationale + answer 0 if you think it is fake.\n Must Keep in mind that the end of a sentence should end with either 0 or 1

###
Output: (rational) + (0||1)
"""

fewshot = """Examples:

Example Article (Real)
Title: "Three women contract HIV from dirty “vampire facials” at unlicensed spa"\\
Content: "Trendy, unproven 'vampire facials' performed at an unlicensed spa in New Mexico left at least three women with HIV infections. 
This marks the first time that cosmetic procedures have been associated with an HIV outbreak, 
according to a detailed report of the outbreak investigation published today. Ars reported on the cluster last year when state health 
officials announced they were still identifying cases linked to the spa despite it being shut down in September 2018. 
But today’s investigation report offers more insight into the unprecedented outbreak, which linked five people with HIV infections to the spa and spurred investigators to contact and test nearly 200 other spa clients."\\
Example Article (Fake)\\
Title: "New NFL Combine Drill Tests Player’s Ability To Half-Ass Taping Of Local Sandwich Shop Commercial"
Content: "INDIANAPOLIS—In an effort to gain a fuller picture of prospective players and ensure they would be good fits for the teams that drafted them,
 a new NFL Scouting Combine drill tested a player’s ability to half-ass a taping of a local sandwich shop commercial,
  sources confirmed Monday. As part of our new drill, invited players have been asked to stand stiffly in a jersey and ill-fitting jeans while reading awkwardly from cue cards that encourage customers to come on down to Big Dave’s or Galaxy Sandwich.\\
  
rationale: When determining the authenticity of a news story, "Three women contract HIV from dirty 'vampire facials' at unlicensed spa" is judged to be real news because it is based on detailed investigative reporting and clear facts from a trusted source 1 . On the other hand, "New NFL Combine Drill Tests Player's Ability To Half-Ass Taping Of Local Sandwich Shop Commercial" is considered fake news because it uses satirical language and an unrealistic claim that the NFL Combine tests players' ability to film commercials 0.  


Now, Answer whether below article is fake or real with format below.
Output: (rational) + (0||1)


"""

In [ ]:
api_key = "your_api_key"
client = OpenAI(api_key=api_key)

In [ ]:
tasks_fewshot = []

# task for cot prompt
for i in range(len(data)):
    task = {
        "custom_id": f"{i}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "temperature": 0,
            "messages": [
                {"role": "system", "content": prompt},
                {"role": "user", "content": fewshot + data[i][0]}
            ]
        }
    }
    tasks_fewshot.append(task)

print(f"The length of tasks_fewshot: {len(tasks_fewshot)}")

In [ ]:
# Creating the file

file_name_fewshot = "batch_fewshot_cot.jsonl"

with open(file_name_fewshot, 'w') as file:
    for obj in tasks_fewshot:
        file.write(json.dumps(obj) + '\n')

In [ ]:
# Uploading the file

batch_file_fewshot = client.files.create(
    file=open(file_name_fewshot, "rb"),
    purpose="batch"
)

In [ ]:
# Creating the batch job
batch_job_fewshot = client.batches.create(
    input_file_id=batch_file_fewshot.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)


In [ ]:
# Checking the status of the batch job
batch_job_fewshot = client.batches.retrieve(batch_job_fewshot.id) # 
client.batches.ret
print(batch_job_fewshot)

## 5. Results

In [ ]:
result_file_fewshot_id = batch_job_fewshot.output_file_id
print(f"Fewshot file id: {result_file_fewshot_id}")

result_fewshot = client.files.content(result_file_fewshot_id).content
result_fewshot_file_name = "data/batch_job_fewshot_results.jsonl"

with open(result_fewshot_file_name, "w") as file:
    file.write(result_fewshot.decode('utf-8'))
    
results_fewshot = []
with open(result_fewshot_file_name, 'r') as file:
    for line in file:
        json_obj = json.loads(line.strip())
        results_fewshot.append(json_obj)


In [ ]:
# Testing whether there is a case that the last character is not 0 or 1
error_case = []

for i in range(len(results_fewshot)):
    res = results_fewshot[i]['response']['body']['choices'][0]['message']['content']
    if res[-1] != '0' and res[-1] != '1':
        if res[-2] != '0' and res[-2] != '1':
            error_case.append(i)

assert len(error_case) == 0